In [13]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, Process
import time

In [16]:
def function(lst):
    print (lst)
    time.sleep(5)
    return lst


In [21]:
%%time
with Pool(processes=8) as p:
    res = p.map(function,[1,2,3,4,5])
    p.close()
    p.join()

1
2
3
5
4
CPU times: user 17.5 ms, sys: 24.1 ms, total: 41.6 ms
Wall time: 5.12 s


In [4]:
import pandas as pd
import lux
from lux.vis.Vis import Vis
from lux.vis.VisList import VisList

In [73]:
# df = pd.read_csv("../../lux-datasets/data/communities.csv")

In [74]:
df = pd.read_csv("../data/airbnb_250x.csv")
df.maintain_metadata()

In [39]:
len(df)

12223750

In [40]:
self = VisList([lux.Clause("?",data_type="quantitative"),lux.Clause("?",data_type="quantitative")])

In [41]:
from lux.processor.Parser import Parser
from lux.processor.Validator import Validator
from lux.processor.Compiler import Compiler
self._inferred_intent = Parser.parse(self._intent)
Validator.validate_intent(self._inferred_intent, df)
self._collection = Compiler.compile_intent(df, self._inferred_intent)

In [42]:
len(self._collection)

20

In [50]:
%%time 
lux.config.executor.execute(self._collection, df)

CPU times: user 17.6 ms, sys: 1.62 ms, total: 19.2 ms
Wall time: 18.8 ms


In [44]:
def execute_vis(vis):
    lux.config.executor.execute([vis],df)

In [45]:
from multiprocessing import Pool, Process

In [51]:
%%time
with Pool(processes=8) as p:
    res = p.map(execute_vis,self._collection)
    p.close()
    p.join()

CPU times: user 6.63 s, sys: 261 ms, total: 6.9 s
Wall time: 7.17 s


In [52]:
%%time
with Pool(processes=5) as p:
    res = p.map(execute_vis,self._collection)
    p.close()
    p.join()

CPU times: user 6.71 s, sys: 262 ms, total: 6.97 s
Wall time: 7.25 s


As expected, simple parallelization is much slower because of the overhead due to serializing/deserializing a big dataframe

In [70]:
# print (self._collection)

In [3]:
import pandas as pd
df = pd.read_csv("../data/airbnb_250x.csv")

In [9]:
def example_heatmap(visdf):
    x_attr = "price"
    y_attr = "minimum_nights"
    visdf["xBin"] = pd.cut(visdf[x_attr], bins=40)
    visdf["yBin"] = pd.cut(visdf[y_attr], bins=40)
    groups = visdf.groupby(["xBin", "yBin"])[x_attr]
    result = groups.count().reset_index(name=x_attr)

    result = result.rename(columns={x_attr: "count"})
    result = result[result["count"] != 0]

    # convert type to facilitate weighted correlation interestingess calculation
    result["xBinStart"] = result["xBin"].apply(lambda x: x.left).astype("float")
    result["xBinEnd"] = result["xBin"].apply(lambda x: x.right)

    result["yBinStart"] = result["yBin"].apply(lambda x: x.left).astype("float")
    result["yBinEnd"] = result["yBin"].apply(lambda x: x.right)

    visdf = result.drop(columns=["xBin", "yBin"])
    return visdf

In [10]:
from multiprocessing import Pool, Process

In [ ]:
%%time
with Pool(processes=8) as p:
    res = p.map(example_heatmap,[df for _ in range(16)])
    p.close()
    p.join()

In [8]:
%%time 
for _ in range(16): example_heatmap("price","minimum_nights",df) 

CPU times: user 773 ms, sys: 119 ms, total: 892 ms
Wall time: 893 ms


,count,xBinStart,xBinEnd,yBinStart,yBinEnd
0,10803250,-10.0,250.0,-0.249,32.225
1,48250,-10.0,250.0,32.225,63.450
2,30000,-10.0,250.0,63.450,94.675
3,11500,-10.0,250.0,94.675,125.900
4,2500,-10.0,250.0,125.900,157.125
...,...,...,...,...,...
1200,250,7500.0,7750.0,-0.249,32.225
1240,250,7750.0,8000.0,-0.249,32.225
1320,250,8250.0,8500.0,-0.249,32.225
1560,1000,9750.0,10000.0,-0.249,32.225


----------------------------------------------------------------------------------------

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/airbnb_100x.csv")

In [4]:
def example_heatmap(visdf):
    x_attr = "price"
    y_attr = "minimum_nights"
    visdf["xBin"] = pd.cut(visdf[x_attr], bins=40)
    visdf["yBin"] = pd.cut(visdf[y_attr], bins=40)
    groups = visdf.groupby(["xBin", "yBin"])[x_attr]
    result = groups.count().reset_index(name=x_attr)

    result = result.rename(columns={x_attr: "count"})
    result = result[result["count"] != 0]

    # convert type to facilitate weighted correlation interestingess calculation
    result["xBinStart"] = result["xBin"].apply(lambda x: x.left).astype("float")
    result["xBinEnd"] = result["xBin"].apply(lambda x: x.right)

    result["yBinStart"] = result["yBin"].apply(lambda x: x.left).astype("float")
    result["yBinEnd"] = result["yBin"].apply(lambda x: x.right)

    visdf = result.drop(columns=["xBin", "yBin"])
    return visdf

In [5]:
%%time
visdf = example_heatmap(df) 

CPU times: user 318 ms, sys: 35.3 ms, total: 354 ms
Wall time: 353 ms


In [7]:
import modin.pandas as pd

In [8]:
df = pd.read_csv("../data/airbnb_100x.csv")

In [10]:
%%time
visdf = example_heatmap(df) 

CPU times: user 2min 7s, sys: 6.05 s, total: 2min 13s
Wall time: 5min 57s


To request implementation, send an email to feature_requests@modin.org.
